In [1]:
import ROOT

Welcome to JupyROOT 6.16/00


In [2]:
file = ROOT.TFile.Open("../output/Charmonium2018B_test/jpsi_charmonium_2018B.root")

mass = ROOT.RooRealVar("mass", "", 2.95, 3.25)
data = ROOT.RooDataSet("data", "", file.Jpsi, ROOT.RooArgSet(mass))


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

[#1] INFO:Eval -- RooAbsReal::attachToTree(mass) TTree Float_t branch mass will be converted to double precision


In [3]:
#Signal PDFs

## Signal PDF: Crystal ball and gaussian

yield_gauss = ROOT.RooRealVar("yield_gauss","", 6e5, 0, 1e6)
mean = ROOT.RooRealVar("mean", "", 3.09, 2.95, 3.25)
sigma_gauss = ROOT.RooRealVar("sigma gauss", "", 0.1, 0.000001, 1)

yield_cb = ROOT.RooRealVar("yield_cb","", 5e5, 0, 1e6)
sigma_cb = ROOT.RooRealVar("sigma cb", "", 0.1, 0.000001, 1)

alpha = ROOT.RooRealVar("alpha", "", 1.4, 0, 10)
n = ROOT.RooRealVar("n", "", 99, 0, 100)
n.setConstant(True)

gauss = ROOT.RooGaussian("gauss", "", mass, mean, sigma_gauss)
crystal_ball = ROOT.RooCBShape("crystal ball", "", mass, mean, sigma_cb, alpha, n)

In [4]:
# Background PDF
yield_back = ROOT.RooRealVar("yield_back", "", 5e4, 0, 1e5)
exp_coef = ROOT.RooRealVar("exp_coef", "", -3, -5, 5)

back_exp = ROOT.RooExponential("back_exp", "", mass, exp_coef)

In [5]:
# Model definition and fitting
model = ROOT.RooAddPdf("model", "", ROOT.RooArgList(gauss, crystal_ball, back_exp),
                       ROOT.RooArgList(yield_gauss, yield_cb, yield_back))

In [ ]:
result = model.fitTo(data, ROOT.RooFit.Extended(True), ROOT.RooFit.Range(2.95, 3.25))

In [ ]:
## Plots

# Colors and styles
colors = {"model" : 2, "signal" : 4, "background" : 3}
styles = {"model" : 1, "signal" : 1, "background" : 2}

# Canvas Definition
can = ROOT.TCanvas("can", "histograms", 850, 650)

# Frame creation
frame = mass.frame(ROOT.RooFit.Title("Dataset: Charmonium 2018B"))
frame.GetXaxis().SetTitle("# \mu^+\mu^- invariant mass [GeV]")

# Data
data.plotOn(frame, ROOT.RooFit.Name("Data"), ROOT.RooFit.DataError(ROOT.RooAbsData.SumW2))

# Model
model.plotOn(frame, ROOT.RooFit.Name("Model"), ROOT.RooFit.LineStyle(styles["model"]), ROOT.RooFit.LineColor(colors["model"]))

# Signal
model.plotOn(frame, ROOT.RooFit.Name("Signal"), ROOT.RooFit.Components(ROOT.RooArgSet(gauss, crystal_ball)), ROOT.RooFit.LineStyle(styles["signal"]), ROOT.RooFit.LineColor(colors["signal"]))

# Background
model.plotOn(frame, ROOT.RooFit.Name("Background"), ROOT.RooFit.Components("back_exp"), ROOT.RooFit.LineStyle(styles["background"]), ROOT.RooFit.LineColor(colors["background"]))

## Legends

leg = ROOT.TLegend(0.7, 0.7, 0.88, 0.89)
leg.AddEntry(frame.findObject("Data"), "Data", "LEP")
leg.AddEntry(frame.findObject("Model"), "Model Fit", "L")
leg.AddEntry(frame.findObject("Signal"), "Signal Fit", "L")
leg.AddEntry(frame.findObject("Background"), "Background fit", "L")

frame.Draw()
leg.Draw("same")

can.Draw()

can.SaveAs("Jpsi_fit.png")